# Emotion Recognition :
> Dataset obtained from kaggle. It contains gray-scale images of different human emotions.

### Task -

> Develope an app which will predict human emotion using webcam


### Importing Liraries :

In [2]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import torch
import torch.nn.functional as F
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.transforms import transforms
from torch.utils.data import DataLoader,Dataset
from torchvision import models
from torch.autograd import Variable
from random import randint
from tqdm.autonotebook import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
plt.ion()   # interactive mode
import warnings
warnings.filterwarnings("ignore")
import cv2

c:\program files\python36\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Loading the data :

In [2]:
data = pd.read_csv('train.csv')
data.head()

,Emotion,Pixels
0,3,221 240 251 254 255 255 255 255 255 255 255 25...
1,6,100 107 108 104 103 113 117 115 120 130 138 14...
2,4,35 50 56 57 63 76 74 79 85 86 105 133 145 152 ...
3,6,119 124 129 135 136 140 142 149 159 156 163 16...
4,2,160 173 186 194 188 185 175 162 153 143 135 12...


### Random Model :

In [3]:
em = np.array(data['Emotion'])

In [4]:
avg_error = 0
for i in range(1000):
    rpred = [randint(0,6) for i in range(len(em))]
    x = em == rpred
    x = sum(x)
    avg_error += x*1.0/len(em)

print(avg_error/1000)

0.14291263762565848


### Preprocessing the data :

In [5]:
# Convert string of pixels to list
def changeTypeToNumpyArray(x):
    x = x.split()
    x = [int(i) for i in x]
    return x

# Reshape the image to (1, 48,48)
def reshapeImagePixels(x):
    x = np.array(x)
    x = x/255
    x = x.reshape(1, 48, 48)
    return x

def preprocess(data):
    data['Pixels'] = data['Pixels'].apply(changeTypeToNumpyArray)
    data['Pixels'] = data['Pixels'].apply(reshapeImagePixels)
    processed_data = data[['Pixels', 'Emotion']]
    return processed_data

In [6]:
# Get Training Data
train_data = preprocess(data)
train_data.head()

,Pixels,Emotion
0,"[[[0.8666666666666667, 0.9411764705882353, 0.9...",3
1,"[[[0.39215686274509803, 0.4196078431372549, 0....",6
2,"[[[0.13725490196078433, 0.19607843137254902, 0...",4
3,"[[[0.4666666666666667, 0.48627450980392156, 0....",6
4,"[[[0.6274509803921569, 0.6784313725490196, 0.7...",2


In [7]:
# Get Test Data 
test_data = pd.read_csv('test.csv')
test_data['Pixels'] = test_data['Pixels'].apply(changeTypeToNumpyArray)
test_data['Pixels'] = test_data['Pixels'].apply(reshapeImagePixels)
test_data.head()

,Pixels
0,"[[[0.592156862745098, 0.592156862745098, 0.592..."
1,"[[[0.33725490196078434, 0.3176470588235294, 0...."
2,"[[[0.1450980392156863, 0.23529411764705882, 0...."
3,"[[[0.4549019607843137, 0.43137254901960786, 0...."
4,"[[[0.396078431372549, 0.4470588235294118, 0.46..."


In [8]:
# Converting to dataset format for pytorch use :
class MyDataset(Dataset):
    def __init__(self, data):
        # Convert data to numpy array.
        self.images = np.array(data['Pixels'])
        self.labels = np.array(data['Emotion'])
        # Convert to Tensors
        print(self.images.shape)
        print(self.labels.shape)
        
    def __getitem__(self, index):
        # Get item at index location
        img = self.images[index]
        # Convert to tensor.
        img = torch.from_numpy(img)
        label = torch.from_numpy(self.labels)
        label = label[index]
        # return tesnor.    
        return (img, label)
    
    def __len__(self):
        return self.images.shape[0]

In [9]:
trainset = MyDataset(train_data)

(4178,)
(4178,)


In [10]:
trainset.__getitem__(0)

(tensor([[[0.8667, 0.9412, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
          [0.8784, 0.9333, 0.9922,  ..., 1.0000, 1.0000, 1.0000],
          [0.8784, 0.9294, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [0.0510, 0.0627, 0.0431,  ..., 0.0706, 0.0667, 0.0863],
          [0.0627, 0.0706, 0.0353,  ..., 0.0941, 0.0510, 0.0706],
          [0.0627, 0.0745, 0.0314,  ..., 0.1686, 0.0745, 0.0627]]],
        dtype=torch.float64), tensor(3))

In [11]:
# Creating a train loader for training :
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

### Creating the Model :

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 5)
        self.max1 = nn.MaxPool2d(3, stride=2)
        self.conv2 = nn.Conv2d(64, 64, 5)
        self.max2 = nn.MaxPool2d(3,stride=2)
        self.conv3 = nn.Conv2d(64, 128, 4)
        self.fc1 = nn.Linear(128 * 5 * 5 , 3072)
        self.fc2 = nn.Linear(3072,7)
        self.fc3 = nn.Softmax()
        
    def forward(self, x):
        x = self.max1(F.relu(self.conv1(x)))
        x = self.max2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.dropout(x)
        x = x.view(-1, 128*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [36]:
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
  (max1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (max2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=3200, out_features=3072, bias=True)
  (fc2): Linear(in_features=3072, out_features=7, bias=True)
  (fc3): Softmax()
)


In [37]:
# Definign the loss and optimizer..
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters())

In [38]:
model = model.cuda()

In [39]:
# Trainign the model
nb_epochs = 100
model.train()
for epoch in range(1, nb_epochs+1):
    train_loss = 0
    i = 0
    for images,label in trainloader:
        images = images.cuda().float()
        label = label.cuda()
        #images = images.float()
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss += loss
        i = i + 1
    print("Epoch :", epoch, " Loss on train set :", train_loss.item()/(i*1.0))

Epoch : 1  Loss on train set : 1.8881825454362475
Epoch : 2  Loss on train set : 1.8787992055179508
Epoch : 3  Loss on train set : 1.8727097838889553
Epoch : 4  Loss on train set : 1.860624939430761
Epoch : 5  Loss on train set : 1.7429446154878339
Epoch : 6  Loss on train set : 1.7107250417461832
Epoch : 7  Loss on train set : 1.6812073219823473
Epoch : 8  Loss on train set : 1.6559555403148856
Epoch : 9  Loss on train set : 1.6066202644173426
Epoch : 10  Loss on train set : 1.5767704621526122
Epoch : 11  Loss on train set : 1.5583376120065004
Epoch : 12  Loss on train set : 1.552309807930284
Epoch : 13  Loss on train set : 1.5432521441510616
Epoch : 14  Loss on train set : 1.537078275025346
Epoch : 15  Loss on train set : 1.5237068001550573
Epoch : 16  Loss on train set : 1.5187154289420324
Epoch : 17  Loss on train set : 1.5252550430880247
Epoch : 18  Loss on train set : 1.520082546554449
Epoch : 19  Loss on train set : 1.5230201546472448
Epoch : 20  Loss on train set : 1.5174134232

In [40]:
img,lab = next(iter(trainloader))

In [41]:
model.eval()
output = model(img.cuda().float())

In [42]:
output = torch.argmax(output, dim=1)
output.shape

torch.Size([32])

In [43]:
output = output.cpu().numpy()
lab = lab.numpy()

In [44]:
# Overall accuracy :
cor = sum(output == lab)
cor/32*100.0

65.625

In [45]:
torch.save(model, 'model_save.pt')

In [46]:
torch.save(model.state_dict(), 'model_save_state.pt')

In [4]:
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
  (max1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (max2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=3200, out_features=3072, bias=True)
  (fc2): Linear(in_features=3072, out_features=7, bias=True)
  (fc3): Softmax()
)


In [5]:
#model.load_state_dict(torch.load('model_save_state.pt'))
#model.eval()

Net(
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1))
  (max1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (max2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=3200, out_features=3072, bias=True)
  (fc2): Linear(in_features=3072, out_features=7, bias=True)
  (fc3): Softmax()
)

### Using opencv :

In [6]:
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
video_capture = cv2.VideoCapture(0)
target = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1)

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2, 5)
        face_crop = frame[y:y + h, x:x + w]
        face_crop = cv2.resize(face_crop, (48, 48))
        face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
        face_crop = face_crop.astype('float32') / 255
        face_crop = face_crop.reshape(1, 1, face_crop.shape[0], face_crop.shape[1])
        
        face_crop = torch.from_numpy(face_crop)
        output = model(face_crop)
        output = torch.argmax(output, dim=1).numpy()
        
        result = target[int(output)]
        cv2.putText(frame, result, (x, y), font, 1, (200, 0, 0), 3, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()